In [ ]:
timm_path = "../input/timm-pytorch-image-models/pytorch-image-models-master"
import sys
sys.path.append(timm_path)
import timm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
from tqdm.notebook import tqdm

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torch import optim

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# pretrained_model from beloved timm
# i am using effb3_ns , you can use your fav model instead
model=timm.create_model('tf_efficientnet_b3_ns', pretrained=False) # set pretrained=True to use the pretrained weights
num_features = model.classifier.in_features
model.classifier = nn.Linear(num_features, 1)

In [ ]:

class Model_3d_2_2d(nn.Module):
    def __init__(self, model , input_channels = 3):
        super().__init__()
        self.model = model
        self.cnn3d = nn.Conv3d( input_channels , 1 , 3)
        self.pool = nn.AdaptiveAvgPool3d((144))
        self.cnn2d = nn.Conv2d(144 , 3 , 3 , stride = 1 , padding = 1)
    
    def forward(self , x):
        x = self.cnn3d(x)
        x = x.squeeze(1)
        x = self.pool(x) #changing Depth , width , height to 144
        x = self.cnn2d(x)
        x = self.model(x)
        
        return x
        

In [ ]:
# let input size be (1 , 3 , 300 ,  256 ,266)
# where channel length is 3 , depth is 300 , height and width is 256
m = Model_3d_2_2d(model ,3 )
ip = torch.randn(1 , 3 ,300 ,  256 ,266)
op = m(ip)
op.shape
